# Background

I want to test how the different algorithm correction methods perform when there is volcanic ash present. My understanding is that volcanic ash, as a strongly absorbing aerosol, will be incorrectly dealt with in the default NASA OBPG processing stream, and that I should use something like Wang and Shi 2007. So, I want to check the performance for a couple of satellite images where there are known ash events and see the difference between the default algorithm and the SWIR/NIR algorithm.

There are a couple of things to think about in terms of differences:

1. The default algo will overcorrect in the blue i.e. the radiance will be too low
2. It's debatable about what will happen for resultant bio-optical algorithms e.g. chl: 
    1. An IOCCG report states that even though the magnitudes of Rrs will be too low, the ratio of blue to green will be the same, therefore the OC4/OC3M flavour of chl algorithms will be OK, but the OCI flavour will not.
    2. Most other sources state that with underestimated reflectances in the blue, the chlorophll concentration will be overestimated.
    
So we need to think about atmospheric correction and chlorophyll algorithms. Unfortunately, I don't have in situ data to actually verify which is "correct"

# Volcanic Ash Events

What satellite data should I use?

Kasatochi 2008, August, day 220

Kamkatcha 2015, September, day 259

## MODIS Files?

A2008221231500: Aug 8th, end of eruption.

A2015259022000: Sept 15th, day of eruption.


# Satellite Data Processing

SeaDAS has a bunch of aerosol subraction models, and one of those is the SWIR/NIR Wang one. But there are others, and in the Forum, Sean suggests trying a couple of the different options. It seems that for at least one of them you can provide an aerosol optical thickness. Which has made me wonder that:

**If you can get AOT from another satellite source, then you can enter it here, and do that atmos correction**

I think this might be the best approach for this project, at least to put in the proposal. Need to do some reading re Verity's work and deriving aerosol optical thickness from e.g. MISR.

# UPDATE

When I did some more reading and digging around, I realised that the above Satellite Data Processing approach isn't going to work, or more precisely, it is impossible with the current way the atmospheric correction processing code is written. 

More specifically, with the current satellites and atmospheric correction algorithms, it is impossible to identify strongly absorbing aerosols. The standard (and non-standard) algorithms detect aerosols based on the signal in the NIR, but unfortunately, using the NIR it is not possible to distinguish if there is a strongly absorbing aerosol or not. A LUT is used to select an aerosol based on the NIR signal and 80 commonly occurring aerosols. 

Even if you know the aerosol type from a different source, including information about aerosol height, there is no way to apply that to only the specific part of the satellite image where that aerosols is present – you would have to apply it to the whole image. *There is no way the current atmospheric correction code can deal with different aerosols unless the default correction is used with the LUT of 80 aerosols. And developing a way to do it would be a whole proposal in itself (with experts that aren’t me involved).*

I'm wondering about a "rough" approach...



# New Data Processing approach

Overview of approach: *Apply a specified aerosol optical depth to a whole satellite image when we know there is plume*

As I see it, there are a couple of ways to do this:
1. Use MISR data:
    - Determine AOD from e.g. MISR
    - Determine how long the plume is present in the atmosphere e.g. from MISR (perhaps the others have an idea about this already)
    - Apply this to THE LEVEL 1 images i.e. to make LEVEL 2
2. Use MODIS data (see details in `LabBook-v1-Kamchatka-MODISApproach`):
    - Determine a mean AOD or angstrom coeff from the day before image
    - Apply this to the LEVEL 1 images i.e. to make LEVEL 2
    
I want to dig into the MISR stuff a little bit

# MISR

Following the details in Verity's paper (https://www.sciencedirect.com/science/article/pii/S0377027316304619), I want to download the data from the Kamchatka remobilization event (Sept 2015) and determine aod. 

I downloaded MINX from: https://github.com/nasa/MINX. The repository also contains lots of documentation, which details where to download data, how to search for data, the data products and using the MINX software. Some of the links in Verity's paper and that documentation don't work... so I ended up going to https://l0dup05.larc.nasa.gov/MISR/cgi-bin/MISR/main.cgi for downloading data and using the instructions in https://github.com/nasa/MINX/blob/master/webdoc/MINX_Doc2.pdf.

## Kamchatka processing details

I searched for:
1. aerosol products (MIL2ASAE)
2. path number 98 (orbit number is 83744)
3. September 2015
4. blocks 43 - 50 (using the subsetting/customization option after the initial search)

I ordered these data (order number 628931181), waiting on response...



I recieved and downloaded the file but I'm having trouble loading it into MINX. I wonder if it's a new file formatting issue, so I've reordered, with an older version of the data included (i.e. a .hdf file)... That new order number is 628979756. I got the new file, it loaded in MINX, but it's hard to locate what is what i.e. where land is...  I've now ordered another file that hopefully has TOA radiances in it, so I can plot that too and figure out what the aot and angstrom data is telling me (order number 629006111) 

It's worth mentioning though, I don't have a MISR image for the day of the eruption, only the day after. So even if I can understand the data in the file it isn't really going to help me.

# UPDATE

Still struggling with MINX and MISR... why not just open the MISR files in python and see what I can figure out? See notebook **`MISR-initial-analysis`**

# UPDATE 2

From the MISR file, I calculated the average angstrom and aod over the plume region (see full details in **`MISR-initial-analysis`**). I then used those values to rerun `l2gen` and try different atmospheric correction methods:

1. Using `aer_opt = -8` and `taua = [0.169, 0.176, 0.181, 0.185, 0.194, 0.197, 0.199, 0.216, 0.221, 0.223, 0.235, 0.255, 0.256, 0.315, 0.373, 0.437]`. File name suffix: `_taua_0-169_0-437`
2. Using `aer_opt = -7 ` and `aer_angstrom = 0.578`

I'm not confident the `aer_opt = -8` option is working correctly - I'm going to check on the Forum. But the fixed angstrom run seems to be reasonable. I'm going to start a new notebook and start looking at those output files: **`LabBook-v3-MODISApproach-take2`**

# AOD UPDATE

I figured out what the problem was with the AOD... I had commented out the `aer_angstrom` value in the line above the `taua` values, and the shell script stopped reading at the comment (rather than continuing past it). So I just had to re-adjust the script slightly and re-run.